In [1]:
using CSV, DataFrames, Dates, JSON, LinearAlgebra, MarketData, Plots, PrettyTables
using Printf, Statistics, TimeSeries

include("common.jl")

load2024 (generic function with 1 method)

In [2]:
dbs = load("D05");

In [10]:
function get_oprice(backday::String, price::Float64, df::DataFrame)::Float64
    df2 = df[backday.<df.Day, :]
    df2 = df2[df2.Dividends.>0, :]

    count = size(df2, 1)
    total_adjustment_factor = 1
    for i in count:-1:1
        row = df2[i, :]
        oclose = row.Close / total_adjustment_factor
        adjustment_factor = oclose / (oclose + row.Dividends)
        total_adjustment_factor = total_adjustment_factor * adjustment_factor
    end

    price / total_adjustment_factor
end

function list_dividend(backday::String, df::DataFrame)
    df2 = df[backday.<df.Day, :]
    df2 = df2[df2.Dividends.>0, :]

    println("Day,Close,Original Close,Dividends,Adjustment Factor")
    count = size(df2, 1)
    total_adjustment_factor = 1
    for i in count:-1:1
        row = df2[i, :]
        oclose = row.Close / total_adjustment_factor
        adjustment_factor = oclose / (oclose + row.Dividends)
        total_adjustment_factor = total_adjustment_factor * adjustment_factor
        println(row.Day, ",", row.Close, ",", oclose, ",", row.Dividends, ",", adjustment_factor)
    end
end

function get_aprice(backday::String, price::Float64, df::DataFrame)::Float64
    df2 = df[backday.<df.Day, :]
    df2 = df2[df2.Dividends.>0, :]

    rprice = price

    count = size(df2, 1)
    total_adjustment_factor = 1
    for i in count:-1:1
        row = df2[i, :]
        oclose = row.Close / total_adjustment_factor
        adjustment_factor = oclose / (oclose + row.Dividends)
        rprice = rprice * adjustment_factor
        total_adjustment_factor = total_adjustment_factor * adjustment_factor
    end

    price * total_adjustment_factor
end

get_aprice (generic function with 1 method)

In [12]:
D8DU = load("D8DU");

In [14]:
list_dividend("2022-03-03", load("A17U"))
# list_dividend("2022-03-03", D8DU)
# get_oprice("2022-03-03", 0.048462, D8DU)


Day,Close,Original Close,Dividends,Adjustment Factor
2023-08-07,2.677179,2.677179,0.01578,0.994140274694119
2023-05-23,2.671326,2.6870715008723733,0.06141,0.9776567533816369
2023-02-09,2.660282,2.7371185187070988,0.07925,0.9718609267666501
2022-08-10,2.698085,2.856389531183477,0.07873,0.97317656089861


In [ ]:
function loadlatest(ticker)
    df = CSV.read("data_latest/" * ticker * ".csv", DataFrame)
    day = SubString.(df.Date, 1, 10)
    d = Date.(day)
    select!(df, Not(:Date))
    df[:, :Date] = d
    df[:, :Day] = day
    df
end